# Predicting Ride Price using NeuralNet Regressor
There are 5 parts in this notebook.

0. Normalizate tne data
1. Training on plaintext data
2. Evaluate model on plaintext data
3. Train and Quantize the Concrete model (Quantization Aware Training)
4. Compile the model to the equivalent FHE circuit
5. Evaluate the FHE model on encrypted data

In [9]:
# import required packages
import time
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor as SklearnMLPRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from concrete.ml.sklearn import NeuralNetRegressor as ConcreteNNRegressor
import torch
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

In [10]:
# import dataset as panda's dataframe
import pandas as pd
taxi_dataset = pd.read_csv("./data/taxi_dataset.csv")

## 0. Normalize the data

In [11]:
scaler = MinMaxScaler()
columns_to_scale = taxi_dataset.columns.difference(['price'])

taxi_dataset[columns_to_scale] = scaler.fit_transform(taxi_dataset[columns_to_scale])
print(taxi_dataset)


        duration  price       tip  passengers  distance  hour_of_day  \
0       0.032663   5.80  0.000000    0.111111  0.018948     0.608696   
1       0.109742  11.80  0.000000    0.222222  0.032249     0.478261   
2       0.016614   5.30  0.000000    0.111111  0.012692     0.826087   
3       0.018083   4.80  0.000000    0.111111  0.012637     0.347826   
4       0.109516  24.36  0.020099    0.111111  0.167064     0.869565   
...          ...    ...       ...         ...       ...          ...   
485841  0.072107  13.80  0.009901    0.222222  0.078692     0.086957   
485842  0.027125   7.56  0.006238    0.111111  0.028456     0.869565   
485843  0.038766   7.55  0.006188    0.333333  0.025674     0.739130   
485844  0.063630  11.75  0.009653    0.111111  0.037486     0.956522   
485845  0.156533  15.80  0.000000    0.111111  0.043669     0.782609   

        hour_of_week  day_of_week  start_location_id  end_location_id  \
0           0.658683     0.666667           0.714286         0

In [12]:
# ConcreteMl currently does not support input or output data in categorical, string, or generic object data types
# so let's convert the data type of the target array to integer
# target = taxi_dataset.price.astype("int")
target = taxi_dataset.price

# print(target)

# split the inputs and targets into a train/test dataset
# TODO: look up random_state parameter
# split the dataset into 80% training data and 20% testing data
X_train, X_test, y_train, y_test = train_test_split(
    taxi_dataset, target, test_size=0.4, random_state=42
)

## 1. Training on plaintext data

In [13]:
# train sklearn MLPRegressor model on the clear
sklearn_MLP_regressor = SklearnMLPRegressor(
                    alpha=1,
                    activation="identity",
                    max_iter=1000,
                    hidden_layer_sizes=(25,),
                    learning_rate_init=0.005,)
sklearn_MLP_regressor.fit(X_train, y_train)

MLPRegressor(activation='identity', alpha=1, hidden_layer_sizes=(25,),
             learning_rate_init=0.005, max_iter=1000)

## 2. Evaluate model on plaintext data

In [14]:
time_begin = time.time()
y_pred = sklearn_MLP_regressor.predict(X_test)
print(y_pred)
execution_time_on_plaintext = (time.time() - time_begin) / len(X_test)
print(f"Model evaluation time on the clear: {execution_time_on_plaintext:.8f} seconds per sample")
# Compute the R2 scores
sklearn_r2_score = r2_score(y_test, y_pred)
print(sklearn_r2_score)

[11.29941699 11.29950719 36.79874836 ... 11.61936247 24.79950316
 12.95937163]
Model evaluation time on the clear: 0.00000024 seconds per sample
0.999999994683011


## 3. Train and Quantize the Concrete model (Quantization Aware Training)

In [15]:
# Instantiate the model with parameter
# TODO: tune the parameter
params_neural_net = {
    # "module__n_w_bits": 6,
    # "module__n_a_bits": 8,
    # "module__n_accum_bits": 16,
    "module__n_hidden_neurons_multiplier": 10,
    "module__n_layers": 2,  # total number of layers in the FCNN = 1 hidden layer
    "module__activation_function": torch.nn.ReLU,
    "max_epochs": 1,
    "verbose": 1,
    # "lr": 0.1,
}


#some sort of Feature preprocessing needed for quantization aware training
# Linear models require polynomial features to be applied before training to fit a non-linear model and other models perform better with this transoformation
pipe = Pipeline(
    [
        ("poly", PolynomialFeatures()),
        ("scaler", StandardScaler()),
    ]
)

X_poly_train = pipe.fit_transform(X_train)
X_poly_test = pipe.transform(X_test)

concrete_NN_regressor = ConcreteNNRegressor(batch_size=32, **params_neural_net)

# train the concrete linear regression model on clear data
# The built-in NN regressor models will automatically quantize weights and activations with .fit() call. (Quantization Aware Training) These models use several layers for Quantization Aware Training, allowing good performance for low precision (down to 2-3 bits) weights and activations.
# The maximum accumulator bit-width is controlled by the number of weights and activation bits, as well as a pruning factor. This factor is automatically determined based on the desired accumulator bit-width and a multiplier factor can be optionally specified.


concrete_NN_regressor.fit(X_poly_train, y_train.values.reshape(-1, 1))
print("done training")

# Now, we can test our Concrete ML model on the clear test data
y_pred_q = concrete_NN_regressor.predict(X_poly_test)
print("done prediction")
# Compute the R2 scores
quantized_r2_score = r2_score(y_test, y_pred_q)

print("done calculating r2 score")

print(quantized_r2_score)

  epoch    train_loss    valid_loss      dur
-------  ------------  ------------  -------
      1       11.7663       40.5754  57.8481
done training
done prediction
done calculating r2 score
0.6659336746804537


## 4. Compile the model to the equivalent FHE circuit

In [16]:
# Compile the quantized model in to FHE circuit and run inference on it
# You have to provide the training dataset in order to compile the quantized model to equivalent FHE circuit
time_begin = time.time()
fhe_circuit = concrete_NN_regressor.compile(X_poly_train)
print(f"Generating a key for a {fhe_circuit.graph.maximum_integer_bit_width()}-bit circuit")
print(f"Compilation time: {time.time() - time_begin:.4f} seconds")

# Compiler returns the circuit, which can be used to generated a secrete key and evaluation key
# secrete key: used for encryption and decryption. only accesible to the client
# evaluation key: used to evaluate the cirucit on encypted data. anyone can access it
print(f"Generating a key for a {fhe_circuit.graph.maximum_integer_bit_width()}-bit circuit")

time_begin = time.time()
fhe_circuit.client.keygen(force=False)
print(f"Key generation time: {time.time() - time_begin:.4f} seconds")

Generating a key for a 12-bit circuit
Compilation time: 248.9701 seconds
Generating a key for a 12-bit circuit
Key generation time: 154.4244 seconds


## 5. Evaluate the FHE model on encrypted data

In [17]:
# Evaluate the FHE-version of the model
time_begin = time.time()
y_pred_fhe = concrete_NN_regressor.predict(X_poly_test[:1], fhe="execute")

execution_time_on_ciphertext = (time.time() - time_begin) / len(X_test)
print(f"Execution time: {execution_time_on_ciphertext:.8f} seconds per sample")
print(f"which is {(execution_time_on_ciphertext / execution_time_on_plaintext):.2f} times slower than prediction on the plaintext data")

: 

In [ ]:
# Measure R2 score of FHE version of the model
fhe_r2_score = r2_score(y_test, y_pred_fhe)

print("R^2 scores:")
print(f"scikit-learn (clear): {sklearn_r2_score:.4f}")
print(f"Concrete ML (quantized model on plaintext): {quantized_r2_score:.4f}")
print(f"Concrete ML (FHE model on ciphertext): {fhe_r2_score:.4f}")

# Measure the error of the FHE quantized model with respect to the clear scikit-learn float model
concrete_score_difference = abs(fhe_r2_score - quantized_r2_score) * 100 / quantized_r2_score
print(
    "\nRelative score difference for Concrete ML (quantized model on clear) vs. Concrete ML (FHE):",
    f"{concrete_score_difference:.2f}%",
)

# Measure the error of the FHE quantized model with respect to the clear float model
score_difference = abs(fhe_r2_score - sklearn_r2_score) * 100 / sklearn_r2_score
print(
    "Relative score difference for scikit-learn (clear) vs. Concrete ML (FHE) scores:",
    f"{score_difference:.2f}%",
)